# Intro to TorchRec

Frequently, when building recommendation systems, we want to represent entities like products or pages with embeddings. For example, see Meta AI's [Deep learning recommendation model](https://arxiv.org/abs/1906.00091), or DLRM. As the number of entities grow, the size of the embedding tables can exceed a single GPU’s memory. A common practice is to shard the embedding table across devices, a type of model parallelism. To that end, **torchRec introduces its primary API called [`DistributedModelParallel`](https://github.com/facebookresearch/torchrec/blob/main/torchrec/distributed/model_parallel.py#L109), or DMP. Like pytorch’s DistributedDataParallel, DMP wraps a model to enable distributed training.**

## **Installation**
Requirements:
- python >= 3.7

We highly recommend CUDA when using torchRec. If using CUDA:
- cuda >= 11.0


In [ ]:
# install conda to make installying pytorch with cudatoolkit 11.3 easier. 
!wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.9.2-Linux-x86_64.sh
!chmod +x Miniconda3-py37_4.9.2-Linux-x86_64.sh
!bash ./Miniconda3-py37_4.9.2-Linux-x86_64.sh -b -f -p /usr/local

In [ ]:
# install pytorch with cudatoolkit 11.3
!pip3 install --pre torch -f https://download.pytorch.org/whl/nightly/cu113/torch_nightly.html

Installing torchRec will also install [FBGEMM](https://github.com/pytorch/fbgemm), a collection of CUDA kernels and GPU enabled operations to run 

In [ ]:
# install torchrec
!pip3 install torchrec-nightly

The following steps are needed for the Colab runtime to detect the added shared libraries. The runtime searches for shared libraries in /usr/lib, so we copy over the libraries which were installed in /usr/local/lib/. **This is a very necessary step, only in the colab runtime**. 

In [ ]:
!cp /usr/local/lib/lib* /usr/lib/

\**Restart your runtime at this point for the newly installed packages to be seen.** Run the step below immediately after restarting so that python knows where to look for packages. **Always run this step after restarting the runtime.**

In [ ]:
import sys
sys.path = ['', '/env/python', '/usr/local/lib/python37.zip', '/usr/local/lib/python3.7', '/usr/local/lib/python3.7/lib-dynload', '/usr/local/lib/python3.7/site-packages']

## **Overview**
This tutorial will cover three pieces of torchRec - the `nn.module` `EmbeddingBagCollection`, the `DistributedModelParallel` API, and the datastructure `KeyedJaggedTensor`.

### Distributed Setup
We setup our environment with torch.distributed. For more info on distributed, see this [tutorial](https://pytorch.org/tutorials/beginner/dist_overview.html)

Here, we use one rank (the colab process) corresponding to our 1 colab GPU.

In [1]:
import os
import torch
import torchrec
import torch.distributed as dist

os.environ["RANK"] = "0"
os.environ["WORLD_SIZE"] = "1"
os.environ["MASTER_ADDR"] = "localhost"
os.environ["MASTER_PORT"] = "29500"

# Note - you will need a V100 or A100 to run tutorial as as!
# If using an older GPU (such as colab free K80), 
# you will need to compile fbgemm with the appripriate CUDA architecture
# or run with "gloo" on CPUs 
dist.init_process_group(backend="nccl")

### From EmbeddingBag to EmbeddingBagCollection
Pytorch represents embeddings through [`torch.nn.Embedding`](https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html) and [`torch.nn.EmbeddingBag`](https://pytorch.org/docs/stable/generated/torch.nn.EmbeddingBag.html). EmbeddingBag is a pooled version of Embedding.

TorchRec extends these modules by creating collections of embeddings. We will use [`EmbeddingBagCollection`](https://github.com/facebookresearch/torchrec/blob/main/torchrec/modules/embedding_modules.py#L59) to represent a group of EmbeddingBags.

Here, we create an EmbeddingBagCollection (EBC) with two embedding bags. Each table, `product_table` and `user_table`, is represented by 64 dimension embedding of size 4096. Note how we initially allocate the EBC on device "meta". This will tell EBC to not allocate memory yet.

In [2]:
ebc = torchrec.EmbeddingBagCollection(
    device="meta",
    tables=[
        torchrec.EmbeddingBagConfig(
            name="product_table",
            embedding_dim=64,
            num_embeddings=4096,
            feature_names=["product"],
            pooling=torchrec.PoolingType.SUM,
        ),
        torchrec.EmbeddingBagConfig(
            name="user_table",
            embedding_dim=64,
            num_embeddings=4096,
            feature_names=["user"],
            pooling=torchrec.PoolingType.SUM,
        )
    ]
)

### DistributedModelParallel
Now, we’re ready to wrap our model with [`DistributedModelParallel`](https://github.com/facebookresearch/torchrec/blob/main/torchrec/distributed/model_parallel.py#L109) (DMP). Instantiating DMP will:

1. Decide how to shard the model. DMP will collect the available ‘sharders’ and come up with a ‘plan’ of the optimal way to shard the embedding table(s) (i.e, the EmbeddingBagCollection)
2. Actually shard the model. This includes allocating memory for each embedding table on the appropriate device(s).

In this toy example, since we have two EmbeddingTables and one GPU, torchRec will place both on the single GPU.


In [3]:
model = torchrec.distributed.DistributedModelParallel(ebc, device=torch.device("cuda"))
print(model)
print(model.plan)

DistributedModelParallel(
  (module): DistributedDataParallel(
    (module): EmbeddingBagCollection(
      (embedding_bags): ModuleDict(
        (product_table): EmbeddingBag(4096, 64, mode=sum)
        (user_table): EmbeddingBag(4096, 64, mode=sum)
      )
    )
  )
)
{'': {'product_table': ParameterSharding(sharding_type='column_wise', compute_kernel='batched_fused', ranks=[0, 0], sharding_spec=EnumerableShardingSpec(shards=[ShardMetadata(shard_offsets=[0, 0], shard_sizes=[4096, 32], placement=rank:0/cuda:0), ShardMetadata(shard_offsets=[0, 32], shard_sizes=[4096, 32], placement=rank:0/cuda:0)])), 'user_table': ParameterSharding(sharding_type='column_wise', compute_kernel='batched_fused', ranks=[0, 0], sharding_spec=EnumerableShardingSpec(shards=[ShardMetadata(shard_offsets=[0, 0], shard_sizes=[4096, 32], placement=rank:0/cuda:0), ShardMetadata(shard_offsets=[0, 32], shard_sizes=[4096, 32], placement=rank:0/cuda:0)]))}}


### Query vanilla nn.EmbeddingBag with input and offsets

We query [`nn.Embedding`](https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html) and [`nn.EmbeddingBag`](https://pytorch.org/docs/stable/generated/torch.nn.EmbeddingBag.html) with `input` and `offsets`. Input is a 1-D tensor containing the lookup values. Offsets is a 1-D tensor where the sequence is a cumulative sum of the number of values to pool per example.

Let's look at an example, recreating the product EmbeddingBag above

```
|------------|
| product ID |
|------------|
| [101, 202] |
| []         |
| [303]      |
|------------|
```



In [4]:
product_eb = torch.nn.EmbeddingBag(4096, 64)
product_eb(input=torch.tensor([101, 202, 303]), offsets=torch.tensor([0, 2, 2]))

tensor([[-2.4314e-01,  2.4152e-01,  1.2018e-01,  9.8066e-01,  7.8772e-01,
         -9.7475e-01,  4.5308e-02,  3.7757e-01,  3.5162e-01, -1.6291e+00,
         -7.1760e-01,  2.5054e-01, -1.5857e-01, -2.8680e-01,  2.3202e-01,
         -5.1505e-01, -2.6178e-01, -3.9254e-02, -3.8008e-01, -7.3679e-01,
          8.0809e-01, -4.3544e-01,  8.3320e-01,  1.1880e-01,  1.4292e-01,
          4.4345e-01, -2.3526e-01, -7.1804e-03,  8.7608e-02,  1.0716e-02,
         -3.7304e-01, -1.4160e+00, -4.4371e-01,  1.7333e+00,  7.9387e-01,
         -1.1216e+00, -5.5665e-01,  1.4938e-01,  4.0931e-01,  1.4251e+00,
          6.5378e-01, -4.9071e-01,  3.3698e-01, -1.5504e-01,  8.9297e-02,
         -7.9015e-01,  1.2470e+00, -1.0940e+00,  9.9707e-01,  8.5721e-01,
          1.4603e-01,  1.0078e+00,  3.2532e-01, -1.1159e+00, -5.4979e-02,
          1.5726e-01, -4.5305e-01, -1.1749e+00,  9.5387e-01,  1.8715e-01,
          7.0722e-01, -2.6525e-01, -1.3195e+00, -6.3983e-01],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0

### Representing minibatches with KeyedJaggedTensor

We need an efficient representation of multiple examples of an arbitrary number of entity IDs per feature per example. In order to enable this "jagged" representation, we use the torchRec datastructure [`KeyedJaggedTensor`](https://github.com/facebookresearch/torchrec/blob/main/torchrec/sparse/jagged_tensor.py#L431) (KJT).

Let's take a look at **how to lookup a collection of two embedding bags**, "product" and "user".  Assume the minibatch is made up of three examples for three users. The first of which has two product IDs, the second with none, and the third with one product ID.

```
|------------|------------|
| product ID | user ID    |
|------------|------------|
| [101, 202] | [404]      |
| []         | [505]      |
| [303]      | [606]      |
|------------|------------|
```

The query should be:

In [5]:
mb = torchrec.KeyedJaggedTensor(
    keys = ["product", "user"],
    values = torch.tensor([101, 202, 303, 404, 505, 606]).cuda(),
    lengths = torch.tensor([2, 0, 1, 1, 1, 1], dtype=torch.int64).cuda(),
)

print(mb.to(torch.device("cpu")))

KeyedJaggedTensor({
    "product": [[101, 202], [], [303]],
    "user": [[404], [505], [606]]
})



Note that the KJT batch size is `batch_size = len(lengths)//len(keys)`. **In the above example, batch_size is 3.**



### Putting it all together, querying our distributed model with a KJT minibatch
Finally, we can query our model using our minibatch of products and users.

The resulting lookup will contain a KeyedTensor, where each key (or feature) contains a 2D tensor of size 3x64 (batch_size x embedding_dim).

In [6]:
pooled_embeddings = model(mb)
print(pooled_embeddings)

KeyedTensor({
    "product": [[-1.8656952381134033, 0.474741131067276, -0.030441880226135254, 0.7665500640869141, -3.278212785720825, -1.0263506174087524, -0.776576042175293, -1.0227174758911133, 0.22396019101142883, 1.4156086444854736, -1.2582138776779175, -0.2592978775501251, -0.12296128273010254, 1.039972186088562, 0.34208089113235474, 1.7708743810653687, -1.6115472316741943, -1.1321913003921509, -0.14693689346313477, -2.68196439743042, -1.308570146560669, 1.4043986797332764, 0.4312652349472046, -0.5712448358535767, 2.8715896606445312, -0.859661877155304, -1.2656745910644531, -2.292487144470215, 0.8639684915542603, 1.5803464651107788, 0.3377319276332855, -0.8498208522796631, -2.493192195892334, 1.321204423904419, -1.5048304796218872, -0.20491421222686768, -0.28394314646720886, -3.1207520961761475, 0.45722758769989014, 1.080879807472229, -0.2185901403427124, 1.2833147048950195, 1.4803493022918701, -0.8197636604309082, 2.287618398666382, -1.3601446151733398, -1.2562706470489502, -0.77

## More resources
For more information, please see our [dlrm](https://github.com/facebookresearch/torchrec/tree/main/examples/dlrm) example, which includes multinode training on the criteo terabyte dataset, using Meta’s [DLRM](https://arxiv.org/abs/1906.00091).